In [1]:
from PIL import Image
import glob
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import load_model, Model
import numpy as np
import json
import os

Using TensorFlow backend.


## Define data path
#### You can add multiple file extensions by extending the glob as shown below 

In [2]:
images_paths = glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA 17, wobbler2x, leaflet2x, display2x, poster gecorrigeerd4x, main banner4x/*.jpg")
images_paths.extend(glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA 17 Ceiling banner/*.jpg"))
images_paths.extend(glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA17 Sensormatic wrap /*.jpg"))
txt_paths = glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA 17, wobbler2x, leaflet2x, display2x, poster gecorrigeerd4x, main banner4x/*.txt")
txt_paths.extend(glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA 17 Ceiling banner/*.txt"))
txt_paths.extend(glob.glob("/home/deeplearning/blob_storage/Classifier1/Klaar MA17 Sensormatic wrap /*.txt"))
                           
file_paths = []
for txt in txt_paths:
    if os.path.getsize(txt) > 0:
        file_paths.append(txt.split('.')[0]+'.jpg')

In [3]:
images_paths = file_paths
print(len(images_paths))

1684


In [4]:
def images_to_sprite(data):
    """
    Creates the sprite image along with any necessary padding
    Source : https://github.com/tensorflow/tensorflow/issues/6322
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data


def populate_img_arr_modified(images_paths, size=(100,100),should_preprocess= False):
    """
    Get an array of images for a list of image paths
    Args:
        size: the size of image , in pixels 
        should_preprocess: if the images should be processed (according to InceptionV3 requirements)
    Returns:
        arr: An array of the loaded images
    """
    arr = []
    arr1 = []
    arr2 = []
    for i,img_path in enumerate(images_paths):
        img = image.load_img(img_path, target_size=size)
        x = image.img_to_array(img)
        x /=255
        x_ = np.zeros((128,128,3))
        y = np.zeros((128,128,3))
        #x = np.expand_dims(x,axis=0)
        arr.append(x_)#np.array([x_,y,x]))
        arr1.append(y)
        arr2.append(x)
    arr = np.array(arr)
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    if should_preprocess:
        arr = preprocess_input(arr)
    return [arr, arr1, arr2]

In [5]:
def images_to_sprite(data):
    """
    Creates the sprite image along with any necessary padding
    Source : https://github.com/tensorflow/tensorflow/issues/6322
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data


def populate_img_arr(images_paths, size=(100,100),should_preprocess= False):
    """
    Get an array of images for a list of image paths
    Args:
        size: the size of image , in pixels 
        should_preprocess: if the images should be processed (according to InceptionV3 requirements)
    Returns:
        arr: An array of the loaded images
    """
    arr = []
    for i,img_path in enumerate(images_paths):
        img = image.load_img(img_path, target_size=size)
        x = image.img_to_array(img)
        arr.append(x)
    arr = np.array(arr)
    if should_preprocess:
        arr = preprocess_input(arr)
    return arr  

## Model Definition
### If you want to use another model, you can change it here

In [11]:
# trained_weights = 'siamese_128_batch64_ep06-0.15.h5'
# model = load_model('../siamese_own_implementation/trained_models/Klaar IN32-1 deel 1 wobbler2x, leaflet2x, display2x, poster4x/' + trained_weights, compile=False)
# model.summary()

trained_weights = 'full_siamese_128_lr_0.003_FC_marg1_batch64_50ep.h5'
model = load_model('../siamese_own_implementation/trained_models/Klaar IN32-1 alles/' + trained_weights , compile=False)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
base_model (Model)              (None, 128)          2705373     input_8[0][0]                    
                                                                 input_9[0][0]                    
          

In [53]:
model.layers[3].get_output_at(1)

<tf.Tensor 'base_model_3/lambda_5/l2_normalize:0' shape=(?, 128) dtype=float32>

In [25]:
new_model = Model(model.inputs, model.layers[-1].output)

AttributeError: Layer base_model has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [12]:
img_arr = populate_img_arr_modified(images_paths,size=(128,128),should_preprocess=False)
preds = model.predict(img_arr,batch_size=64)[2]
preds.reshape(1546,21025).tofile("./oss_data/ma17_only_test_fullmodel.bytes")
#del img_arr,preds

ValueError: cannot reshape array of size 215552 into shape (1546,21025)

In [18]:
preds.shape

(1684, 128)

In [17]:
preds.tofile("./oss_data/ma17_only_test_fullmodel.bytes")



In [62]:
preds.reshape(1546,21025).tofile("./oss_data/ma17_only_test_fullmodel.bytes")

In [6]:
raw_imgs = populate_img_arr(images_paths ,size=(32,32),should_preprocess=False)
sprite = Image.fromarray(images_to_sprite(raw_imgs).astype(np.uint8))
sprite.save('./oss_data/ma17_only_test_fullmodel_smallimg.png')
#del raw_imgs

In [44]:
#raw_imgs = populate_img_arr(images_paths ,size=(128,128),should_preprocess=False)
oss_json = json.load(open('./oss_data/oss_demo_projector_config.json'))
tensor_shape = [raw_imgs.shape[0], model.output_shape[1]]
single_image_dim = [raw_imgs.shape[1], raw_imgs.shape[2]]

json_to_append = {"tensorName": 'Final_visual',
                  "tensorShape": tensor_shape,
                  "tensorPath": "./oss_data/" + 'ma17_only_test.bytes',
                  "sprite": {"imagePath": "./oss_data/" + 'ma17_only_test.png',
                             "singleImageDim": single_image_dim}}
oss_json['embeddings'].append(json_to_append)
with open('oss_data/oss_demo_projector_config.json', 'w+') as f:
    json.dump(oss_json, f, ensure_ascii=False, indent=4)
print('Done')

Done
